<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Women-in-the-House-of-Representatives" data-toc-modified-id="Women-in-the-House-of-Representatives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Women in the House of Representatives</a></span><ul class="toc-item"><li><span><a href="#Read-table-from-wikipedia" data-toc-modified-id="Read-table-from-wikipedia-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Read table from wikipedia</a></span></li><li><span><a href="#Go-row-by-row-and-extract-information-about-each-rep" data-toc-modified-id="Go-row-by-row-and-extract-information-about-each-rep-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Go row by row and extract information about each rep</a></span></li><li><span><a href="#In-order-to-visualise-the-data-like-below,-we-need-to-sort-the-rep-terms-appropriately" data-toc-modified-id="In-order-to-visualise-the-data-like-below,-we-need-to-sort-the-rep-terms-appropriately-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>In order to visualise the data like below, we need to sort the rep terms appropriately</a></span></li><li><span><a href="#Save-rep-data-to-a-csv-to-be-plotted-in-the-visualisation-later" data-toc-modified-id="Save-rep-data-to-a-csv-to-be-plotted-in-the-visualisation-later-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Save rep data to a csv to be plotted in the visualisation later</a></span></li></ul></li><li><span><a href="#Women-in-the-Senate" data-toc-modified-id="Women-in-the-Senate-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Women in the Senate</a></span><ul class="toc-item"><li><span><a href="#Go-row-by-row-and-extract-information-about-each-senator" data-toc-modified-id="Go-row-by-row-and-extract-information-about-each-senator-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Go row by row and extract information about each senator</a></span></li><li><span><a href="#In-order-to-visualise-the-data,-we-need-to-sort-the-senator-terms-appropriately" data-toc-modified-id="In-order-to-visualise-the-data,-we-need-to-sort-the-senator-terms-appropriately-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>In order to visualise the data, we need to sort the senator terms appropriately</a></span></li><li><span><a href="#Save-data-to-a-csv-to-be-plotted-in-the-visualisation-later" data-toc-modified-id="Save-data-to-a-csv-to-be-plotted-in-the-visualisation-later-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Save data to a csv to be plotted in the visualisation later</a></span></li></ul></li><li><span><a href="#Total-number-of-women-serving-over-time" data-toc-modified-id="Total-number-of-women-serving-over-time-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Total number of women serving over time</a></span></li><li><span><a href="#Data-for-all-congressmen-and-women" data-toc-modified-id="Data-for-all-congressmen-and-women-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Data for all congressmen and women</a></span></li></ul></div>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

## Women in the House of Representatives

### Read table from wikipedia

In [187]:
req = requests.get("https://en.wikipedia.org/wiki/Women_in_the_United_States_House_of_Representatives")

soup = BeautifulSoup(req.content, 'lxml')
wikitables = soup.findAll("table", {"class": ["wikitable"]})

### Go row by row and extract information about each rep

In [188]:
def extract_info(row):
    img = None
    name = None
    party = None
    district = None
    name_wiki = None
    if len(row.findAll("td")) == 6:
        img, name, party, district, time_served, notes = row.findAll("td")
    elif len(row.findAll("td")) == 3:
        district, time_served, notes = row.findAll("td")
    elif len(row.findAll("td")) == 2:
        time_served, notes = row.findAll("td")
    else:
        return None

    if img != None:
        img = img.find("a")["href"]

    if name != None:
        name_wiki = name.find("a")["href"]
        name = name.find("a").text
        
    if party != None:
        party = party.find("a").text

    if district != None:
        district = district.find("a").text

    try:
        term_start = pd.to_datetime(list(filter(lambda x: "," in x.text, time_served.findAll("span")))[0].text)
    except:
        if len(time_served.findAll("span")) == 3:
            term_start = pd.to_datetime(time_served.findAll("span")[0].findAll("span")[-1].text)

    term_end = time_served.text.split("\n")[-1]
    #print(term_end)
    if term_end == "present":
        term_end = pd.to_datetime("2020-01-01") # Change to latest possible term end (could be 2018 for some reps...)
    else:
        try:
            term_end = pd.to_datetime(term_end)
        except:
            term_end = pd.to_datetime(time_served.findAll("span")[-1].text)
    notes = notes.text
    
    return {"image":img, "name_wiki":name_wiki,
            "name":name, "party":party, "district":district,
            "term_start":term_start, "term_end":term_end, "notes":notes}

In [189]:
# Map over each row in the table, extracting info about each rep
df = list(map(extract_info, wikitables[2].findAll("tr")))

# Convert list into a dataframe, removing empty rows and filling empty cells based on data in the row above
women_reps = pd.DataFrame.from_records(filter(None, df)).fillna(method="pad")

# Make a clean version of the rep's name, removing punctuation and underscores and making it lowercaase
women_reps["clean_name"] = women_reps["name_wiki"].str.replace("/wiki/", "").str.replace('[^\w\s]','').str.replace("_", "").str.lower()

# Sort according to the rep's start date
women_reps = women_reps.sort_values("term_start")

### In order to visualise the data like below, we need to sort the rep terms appropriately

![](https://i.imgur.com/gc5yGMv.png)

In [228]:
def decimal_date(date):
    """Return a decimal version of the date"""
    return date.year + (date.dayofyear-1)/365.25

# Initialise array to store stream data
streams = []
stream_list = []

# Iterate over each rep
for i, row in women_reps.iterrows():
    # Start at the bottom
    streams.append(0)
    # Loop through each stream currently stored
    for stream_id, stream in enumerate(streams):
        # If the start date of the rep's term is greater than the date stored for this stream
        if decimal_date(row.term_start) >= stream:
            # We have found the right spot to put the rep's line
            streams[stream_id] = decimal_date(row.term_end)
            stream_list.append(stream_id+1)
            break
        # otherwise, continue looping

# Finally, add the stream data to the dataframe
women_reps["stream"] = stream_list

In [229]:
women_reps.head()

,district,image,name,name_wiki,notes,party,term_end,term_start,clean_name,stream
0,Montana at-large,/wiki/File:Jeannette_Rankin_cph.3b13863.jpg,Jeannette Rankin,/wiki/Jeannette_Rankin,First woman elected to a national office\nReti...,Republican,1919-03-04,1917-03-04,jeannetterankin,1
2,Oklahoma's 2nd,/wiki/File:A.M._Robertson.jpg,Alice Robertson,/wiki/Alice_Mary_Robertson,First woman to defeat an incumbent congressman...,Republican,1923-03-04,1921-03-04,alicemaryrobertson,1
3,Illinois at-large,/wiki/File:Mrs-winifred-huck.jpg,Winnifred Huck,/wiki/Winnifred_Sprague_Mason_Huck,Succeeded her father in a special election\nFi...,Republican,1923-03-04,1922-11-07,winnifredspraguemasonhuck,2
4,California's 5th,/wiki/File:Mae_Nolan.jpg,Mae Nolan,/wiki/Mae_Nolan,Succeeded her husband\nRetired,Republican,1925-03-04,1923-01-23,maenolan,3
5,California's 4th,/wiki/File:Florence_Prag_Kahn.jpg,Florence Kahn,/wiki/Florence_Prag_Kahn,First woman to be reelected\nFirst Jewish woma...,Republican,1937-01-03,1925-03-04,florencepragkahn,1


### Save rep data to a csv to be plotted in the visualisation later

In [230]:
women_reps[["name", "district", "term_start", "term_end", "party", "clean_name", "stream"]].reset_index().rename(columns={"index":"id"}).to_csv("women_reps.csv", index=False)

## Women in the Senate

In [407]:
req = requests.get("https://en.wikipedia.org/wiki/Women_in_the_United_States_Senate")

soup = BeautifulSoup(req.content, 'lxml')
wikitables = soup.findAll("table", {"class": ["wikitable"]})

### Go row by row and extract information about each senator

In [416]:
def extract_info(row):
    if len(row.findAll("td")) == 9:
        img, name, state, term_start, term_end, placeholder, placeholder, placeholder, party = row.findAll("td")
    else:
        return None

    img = img.find("a")["href"]

    name_wiki = name.find("a")["href"]
    name = name.find("a").text
        
    party = party.find("a").text
    state = state.find("a").text
    term_start = pd.to_datetime(term_start.findAll("span")[-1].text)
    
    try:
        term_end = term_end.findAll("span")[-1].text
    except IndexError:
        term_end = term_end.text
    
    if term_end == "present":
        term_end = pd.to_datetime("2020-01-01") # Change to latest possible term end (could be 2018 for some reps...)
    else:
        term_end = pd.to_datetime(term_end.replace("—", ""))
    
    return {"image":img, "name_wiki":name_wiki,
            "name":name, "party":party, "state":state,
            "term_start":term_start, "term_end":term_end}

In [417]:
# Map over each row in the table, extracting info about each rep
df = list(map(extract_info, wikitables[2].findAll("tr")))

# Convert list into a dataframe, removing empty rows and filling empty cells based on data in the row above
women_senators = pd.DataFrame.from_records(filter(None, df)).fillna(method="pad")

# Make a clean version of the rep's name, removing punctuation and underscores and making it lowercaase
women_senators["clean_name"] = women_senators["name_wiki"].str.replace("/wiki/", "").str.replace('[^\w\s]','').str.replace("_", "").str.lower()

# Sort according to the rep's start date
women_senators = women_senators.sort_values("term_start")

### In order to visualise the data, we need to sort the senator terms appropriately

In [426]:
def decimal_date(date):
    """Return a decimal version of the date"""
    return date.year + (date.dayofyear-1)/365.25

# Initialise array to store stream data
streams = []
stream_list = []

# Iterate over each rep
for i, row in women_senators.iterrows():
    # Start at the bottom
    streams.append(0)
    # Loop through each stream currently stored
    for stream_id, stream in enumerate(streams):
        # If the start date of the rep's term is greater than the date stored for this stream
        if decimal_date(row.term_start) >= stream:
            # We have found the right spot to put the rep's line
            streams[stream_id] = decimal_date(row.term_end)
            stream_list.append(stream_id+1)
            break
        # otherwise, continue looping

# Finally, add the stream data to the dataframe
women_senators["stream"] = stream_list

In [427]:
women_senators.head()

,image,name,name_wiki,party,state,term_end,term_start,clean_name,stream
0,/wiki/File:Reb_Felton-Geo_Senate.jpg,Rebecca Felton,/wiki/Rebecca_Latimer_Felton,Democratic,Georgia,1922-11-22,1922-11-21,rebeccalatimerfelton,1
1,/wiki/File:Hattie_Caraway_1914.jpg,Hattie Caraway,/wiki/Hattie_Caraway,Democratic,Arkansas,1945-01-03,1931-12-09,hattiecaraway,1
2,/wiki/File:RoseLong.jpg,Rose Long,/wiki/Rose_McConnell_Long,Democratic,Louisiana,1937-01-03,1936-01-31,rosemcconnelllong,2
3,/wiki/File:SenatorDixieBibbGraves.jpg,Dixie Graves,/wiki/Dixie_Bibb_Graves,Democratic,Alabama,1938-01-10,1937-08-20,dixiebibbgraves,2
4,/wiki/File:Gladys_Pyle.jpg,Gladys Pyle,/wiki/Gladys_Pyle,Republican,South Dakota,1939-01-03,1938-11-09,gladyspyle,2


### Save data to a csv to be plotted in the visualisation later

In [428]:
women_senators[["name", "state", "term_start", "term_end", "party", "clean_name", "stream"]]\
    .reset_index()\
    .rename(columns={"index":"id"})\
    .to_csv("women_senators.csv", index=False)

## Total number of women serving over time

In [794]:
def get_total_number_of_women(df, date):
    
    row_dict = df.loc[(date > df.term_start) & (date <= df.term_end)].groupby("party").count()["clean_name"].to_dict()
    row_dict["date"] = date
    
    return row_dict

num_women = pd.DataFrame(list(map(lambda x: get_total_number_of_women(women_reps, x),
                                  pd.date_range(start="1915-03-01", end="2019-03-01", freq="6MS"))))

num_women = num_women.rename(columns={"Democratic":"dem_reps", "Republican":"rep_reps"}).fillna(0)

In [795]:
num_women = pd.DataFrame(list(map(lambda x: get_total_number_of_women(women_senators, x), num_women["date"])))\
    .fillna(0)\
    .rename(columns={"Democratic":"dem_sens", "Republican":"rep_sens"})\
    .join(num_women.set_index("date"), on="date")

num_women[["dem_reps", "rep_reps", "dem_sens", "rep_sens"]] = num_women[["dem_reps", "rep_reps", "dem_sens", "rep_sens"]].astype(int)

num_women = num_women.set_index("date")

In [796]:
# Calculate totals
num_women["total_women_sens"] = num_women["dem_sens"] + num_women["rep_sens"]
num_women["total_women_reps"] = num_women["dem_reps"] + num_women["rep_reps"]

In [797]:
# Save to csv
num_women.to_csv("number_women_over_time.csv")

## Data for all congressmen and women

Download data of all congressmen and women in history

In [657]:
from pandas.io.json import json_normalize

congress_url = requests.get("https://theunitedstates.io/congress-legislators/legislators-historical.json").json()
congress_url.extend(requests.get("https://theunitedstates.io/congress-legislators/legislators-current.json").json())
congress_members = json_normalize(congress_url)[[ "name.official_full", "bio.birthday", "bio.gender", "id.wikidata", "id.thomas", "id.wikipedia", "name.first", "name.last", "terms"]]

Create a single row for each member-term

In [658]:
# Pretty inefficient to be honest but doesn't take more than a minute
def flatten_nested_df(congress_member):
    return pd.DataFrame(congress_member[1]["terms"]).apply(lambda row: row.append(congress_member[1].drop("terms")), axis=1)#.to_dict("records")

from multiprocessing import Pool
pool = Pool(16)

congress_members = pd.concat(list(pool.map(flatten_nested_df, congress_members.iterrows())))

In [659]:
# Drop unneeded data
congress_members = congress_members.drop(["fax", "office", "contact_form", "address", "phone", "party_affiliations"], axis=1)
# Set start date as type date
congress_members["start"] = pd.to_datetime(congress_members["start"])

In [660]:
# Filter out any members that served before 1915
congress_members = congress_members.query("start > '1915-01-01'")

congress_members = congress_members.rename(columns={"bio.birthday":"dob", "bio.gender":"gender", "id.thomas":"thomas_id",
                                 "id.wikidata":"wikidata_id", "name.first":"first_name", "name.last":"last_name",
                                "name.official_full": "full_name", "start":"term_start", "end":"term_end"})

congress_members["gender"] = congress_members["gender"].astype("category")
congress_members["type"] = congress_members["type"].astype("category")
congress_members["term_end"] = pd.to_datetime(congress_members["term_end"])
congress_members["dob"] = pd.to_datetime(congress_members["dob"])

In [693]:
# Save to HDF
congress_members.to_hdf("list_of_members.h5", "members", format="table")

Get total number of members in congress by party and presidential term

Source: https://en.wikipedia.org/wiki/Party_divisions_of_United_States_Congresses

In [785]:
num_members_ = """date	total_sens	dem_sens	rep_sens	other_sens	total_reps	dem_reps	rep_reps	other_reps
1913-04-03	96	51	44	1	435	290	127	18
1913-04-03	96	56	39	1	435	231	193	8
1913-04-03	96	53	42	1	435	210	216	9
1913-04-03	96	47	48	1	435	191	237	7
1921-04-03	96	37	59	0	435	132	300	1
1923-02-08	96	43	51	2	435	207	225	3
1923-02-08	96	40	54	1	435	183	247	5
1923-02-08	96	47	48	1	435	195	237	3
1929-04-03	96	39	56	1	435	163	267	1
1929-04-03	96	47	48	1	435	217	217	1
1933-04-03	96	59	36	1	435	313	117	5
1933-04-03	96	69	25	2	435	322	103	10
1933-04-03	96	75	17	4	435	333	89	13
1933-04-03	96	69	23	4	435	262	169	4
1933-04-03	96	66	28	2	435	267	162	6
1933-04-03	96	57	38	1	435	222	209	4
1945-12-04	96	57	38	1	435	243	190	2
1945-12-04	96	45	51	0	435	188	246	1
1945-12-04	96	54	42	0	435	263	171	1
1945-12-04	96	48	47	1	435	234	199	2
1953-01-20	96	46	48	2	435	213	221	1
1953-01-20	96	48	47	1	435	232	203	0
1953-01-20	96	49	47	0	435	234	201	0
1953-01-20	98	64	34	0	437	283	153	1
1961-01-20	100	64	36	0	437	262	175	0
1963-11-22	100	67	33	0	435	258	176	0
1963-11-22	100	68	32	0	435	295	140	0
1963-11-22	100	64	36	0	435	247	187	0
1969-01-20	100	58	42	0	435	243	192	0
1969-01-20	100	54	44	2	435	255	180	0
1974-09-08	100	56	42	2	435	242	192	1
1974-09-08	100	61	37	2	435	291	144	0
1977-01-20	100	61	38	1	435	292	143	0
1977-01-20	100	58	41	1	435	277	158	0
1981-01-20	100	46	53	1	435	242	192	1
1981-01-20	100	46	54	0	435	269	166	0
1981-01-20	100	47	53	0	435	253	182	0
1981-01-20	100	55	45	0	435	258	177	0
1989-01-20	100	55	45	0	435	260	175	0
1989-01-20	100	56	44	0	435	267	167	1
1993-01-20	100	57	43	0	435	258	176	1
1993-01-20	100	47	53	0	435	204	230	1
1993-01-20	100	45	55	0	435	207	226	2
1993-01-20	100	45	55	0	435	211	223	1
2001-01-20	100	50	50	0	435	212	221	2
2001-01-20	100	48	51	1	435	205	229	1
2001-01-20	100	44	55	1	435	202	231	1
2001-01-20	100	49	49	2	435	236	199	0
2009-01-20	100	57	41	2	435	257	178	0
2009-01-20	100	51	47	2	435	193	242	0
2009-01-20	100	53	45	2	435	201	234	0
2009-01-20	100	44	54	2	435	188	247	0
2017-01-20	100	46	52	2	435	194	241	0
2021-01-20	100	46	52	2	435	194	241	0"""

from io import StringIO
num_members = pd.read_csv(StringIO(num_members_), sep="\t")

num_members.groupby("date").mean().astype(int).to_csv("total_members_over_time.csv")

Do the same for Presidential info

In [750]:
presidents_csv = """president,term_start
Woodrow Wilson,1913-04-03
Warren G. Harding,1921-04-03
Calvin Coolidge,1923-02-08
Herbert Hoover,1929-04-03
Franklin D. Roosevelt,1933-04-03
Harry S. Truman,1945-12-04
Dwight D. Eisenhower,1953-01-20
John F. Kennedy,1961-01-20
Lyndon B. Johnson,1963-11-22
Richard Nixon,1969-01-20
Gerald Ford,1974-09-08
Jimmy Carter,1977-01-20
Ronald Reagan,1981-01-20
George H. W. Bush,1989-01-20
Bill Clinton,1993-01-20
George W. Bush,2001-01-20
Barack Obama,2009-01-20
Donald Trump,2017-01-20
,2021-01-20"""

from io import StringIO
presidents = pd.read_csv(StringIO(presidents_csv))

presidents["term_start"] = pd.to_datetime(presidents["term_start"])